In [2]:
import pystac_client
import planetary_computer
import pandas as pd
import datetime

# 1. Corregimos el warning y configuramos el catálogo
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
def buscar_pares_entrenamiento(bioma_nombre, coords, n_pares=50, dias_margen=30):
    buffer = 0.1
    bbox = [coords[0]-buffer, coords[1]-buffer, coords[0]+buffer, coords[1]+buffer]
    
    search_cloudy = catalog.search(
        collections=["sentinel-2-l2a"],
        bbox=bbox,
        datetime="2023-01-01/2023-12-31",
        query={"eo:cloud_cover": {"gt": 20, "lt": 60}}, 
        max_items=n_pares
    )
    cloudy_items = list(search_cloudy.items())

    pares = []
    for c_item in cloudy_items:
        # Extraer fecha de la imagen nublada
        fecha_nublada = c_item.datetime
        fecha_inicio = (fecha_nublada - datetime.timedelta(days=dias_margen)).strftime('%Y-%m-%dT%H:%M:%SZ')
        fecha_fin = (fecha_nublada + datetime.timedelta(days=dias_margen)).strftime('%Y-%m-%dT%H:%M:%SZ')

        # B. Buscamos la imagen limpia en ese rango específico de ±30 días
        search_clear = catalog.search(
            collections=["sentinel-2-l2a"],
            bbox=c_item.bbox,
            datetime=f"{fecha_inicio}/{fecha_fin}",
            query={"eo:cloud_cover": {"lt": 5}}, 
            max_items=1
        )
        clear_items = list(search_clear.items())
        
        if clear_items:
            pares.append({
                "bioma": bioma_nombre,
                "id_nublada": c_item.id,
                "id_limpia": clear_items[0].id,
                "nubes_porcentaje": c_item.properties["eo:cloud_cover"],
                "item_nublado": c_item,
                "item_limpio": clear_items[0]
            })
    return pares



In [ ]:
biomas = {
    "bosque": [-84.0, 10.3],
    "ciudad": [-74.0, 40.7],
    "desierto": [25.0, 25.0],
    "tundra": [68.0, 70.0],
    "sabana": [34.0, -1.3],
    "manglar": [-80.0, 0.5],
    "montaña": [86.9, 27.9],
    "pradera": [-100.0, 44.0],
    "humedal": [-58.0, -34.5],
    "agricultura": [-3.0, 40.0],
    "glaciar": [-72.0, -50.0],
    "volcanico": [-155.0, 19.4],
    "costa": [-77.0, 24.5],
    "isla_tropical": [-60.0, 14.0]
}

dataset_pares = []

for nombre, coords in biomas.items():
    print(f"Buscando pares para {nombre}...")
    dataset_pares.extend(buscar_pares_entrenamiento(nombre, coords, n_pares=100))

df_pares = pd.DataFrame(dataset_pares).drop(columns=['item_nublado', 'item_limpio'])
print(f"\nTotal de pares encontrados: {len(df_pares)}")
display(df_pares.head())


# Ejemplo: Analizar el primer par encontrado
ejemplo = dataset_pares[0]['item_nublado']
props = ejemplo.properties

print(f"Análisis del item: {ejemplo.id}")
print(f"- Nubes totales: {props.get('s2:cloud_shadow_percentage')}%")
print(f"- Nubes Cirrus (finas): {props.get('s2:thin_cirrus_percentage')}%")
print(f"- Sombras: {props.get('s2:cloud_shadow_percentage')}%")

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import rioxarray
import numpy as np
import torchvision.transforms.functional as TF
import random

class CloudSegmentationDataset(Dataset):
    def __init__(self, df_pares, patch_size=256, train=True):
        self.df = df_pares
        self.patch_size = patch_size
        self.train = train
        self.bands = ["B04", "B03", "B02", "B08", "B11", "B12"]

    def __len__(self):
        return len(self.df)

    def transform(self, image, mask, target):
        # Image y Target son tensores (C, H, W). Mask es (1, H, W)
        if self.train:
            # Flip horizontal
            if random.random() > 0.5:
                image = TF.hflip(image)
                mask = TF.hflip(mask)
                target = TF.hflip(target)

            # Rotaciones de 90 grados
            angle = random.choice([0, 90, 180, 270])
            if angle != 0:
                image = TF.rotate(image, angle)
                mask = TF.rotate(mask, angle)
                target = TF.rotate(target, angle)

        return image, mask, target

    def _get_patch(self, item, is_mask=False):
        signed_item = planetary_computer.sign(item)
        if is_mask:
            url = signed_item.assets["SCL"].href
            da = rioxarray.open_rasterio(url)
            patch = da.isel(y=slice(0, self.patch_size), x=slice(0, self.patch_size)).values
            mask = np.isin(patch, [3, 8, 9, 10]).astype(np.float32)
            return torch.from_numpy(mask) # Retorna (1, H, W) o (H, W)
        else:
            band_data = []
            for b in self.bands:
                url = signed_item.assets[b].href
                da = rioxarray.open_rasterio(url)
                patch = da.isel(y=slice(0, self.patch_size), x=slice(0, self.patch_size)).values
                # Normalización robusta por banda: (x - min) / (max - min) aproximado
                # Sentinel-2 escala 0-10000, pero raramente pasa de 4000 en reflectancia
                patch = np.clip(patch / 4000.0, 0, 1)
                band_data.append(patch)

            img = np.concatenate(band_data, axis=0).astype(np.float32)
            return torch.from_numpy(img)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_input = self._get_patch(row['item_nublado'], is_mask=False)
        mask = self._get_patch(row['item_nublado'], is_mask=True)
        img_target = self._get_patch(row['item_limpio'], is_mask=False)

        # Aplicar aumentos de datos de forma consistente a los 3 elementos
        img_input, mask, img_target = self.transform(img_input, mask, img_target)

        return {
            "input": img_input,   # Para Modelo A y B
            "mask": mask,         # Target para Modelo A, Input para Modelo B
            "target": img_target  # Target para Modelo B
        }

# Crear el DataLoader
train_ds = CloudSegmentationDataset(df_pares)
train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import rioxarray
import planetary_computer
from rasterio.enums import Resampling

def visualizar_comparativa(par, patch_size=512):
    # 1. Firmar los items para obtener acceso a los datos
    item_nublado = planetary_computer.sign(par['item_nublado'])
    item_limpio = planetary_computer.sign(par['item_limpio'])
    
    # 2. Función unificada para extraer parches alineados
    def get_data_aligned(item, is_scl=False):
        bands = ["B04", "B03", "B02"] if not is_scl else ["SCL"]
        data = []
        
        # Usamos la banda B04 (10m) como referencia espacial absoluta
        with rioxarray.open_rasterio(item.assets["B04"].href) as ref:
            # Calculamos el centro basado en la resolución de 10m
            cy, cx = ref.shape[1] // 2, ref.shape[2] // 2
            # Definimos el área de recorte de referencia
            ref_patch = ref.isel(y=slice(cy, cy+patch_size), x=slice(cx, cx+patch_size))

        for b in bands:
            with rioxarray.open_rasterio(item.assets[b].href) as da:
                # Si es SCL (20m), re-muestreamos para que encaje en la rejilla de 10m
                if is_scl:
                    # Usamos Resampling.nearest (0) para no inventar clases intermedias
                    da = da.rio.reproject_match(ref_patch, resampling=Resampling.nearest)
                
                # Extraemos el parche usando los mismos índices centrales
                patch = da.isel(y=slice(cy, cy+patch_size), x=slice(cx, cx+patch_size)).values
                data.append(patch)
        
        if is_scl:
            # SCL devuelve (1, H, W), tomamos la primera capa
            return data[0][0] 
        
        # Para RGB: Stack canales y transponer a (H, W, C)
        # Sentinel-2 se visualiza bien normalizando entre 0 y 3000-4000
        rgb = np.concatenate(data, axis=0).transpose(1, 2, 0)
        return np.clip(rgb / 3500, 0, 1)

    # 3. Carga de datos procesados
    print(f"Procesando visualización para: {par['bioma']}...")
    img_nublada = get_data_aligned(item_nublado, is_scl=False)
    img_limpia = get_data_aligned(item_limpio, is_scl=False)
    mask_scl = get_data_aligned(item_nublado, is_scl=True)

    # 4. Crear la visualización
    fig, ax = plt.subplots(1, 3, figsize=(18, 6))
    
    ax[0].imshow(img_nublada)
    ax[0].set_title(f"Input: Nublado ({par['nubes_porcentaje']:.1f}%)")
    
    ax[1].imshow(img_limpia)
    ax[1].set_title("Target: Limpio (Ground Truth)")
    
    # Visualización binaria de la máscara para el Modelo A
    # Clases SCL: 3 (sombras), 8 (nubes media prob), 9 (alta prob), 10 (cirrus)
    mask_binaria = np.isin(mask_scl, [3, 8, 9, 10])
    ax[2].imshow(mask_binaria, cmap='gray')
    ax[2].set_title("Máscara de Nubes/Sombras")
    
    for a in ax:
        a.axis('off')

    plt.tight_layout()
    plt.show()

# Ejecución
visualizar_comparativa(dataset_pares[900])

In [ ]:
# Verifica que cada fila tenga un ID distinto pero cercano
print(df_pares[['id_nublada', 'id_limpia', 'nubes_porcentaje']].head(10))

# Cuenta cuántos pares tienes por cada tipo de bioma
print(df_pares['bioma'].value_counts())